In [1]:

DATA_PATH = "/Users/luca/Library/CloudStorage/GoogleDrive-luca@ucsc.edu/Shared drives/Birds and CS/Data/CA-Final"


In [2]:
import numpy as np
import os
import sys
from scgt import GeoTiff

In [3]:
sys.path.append(os.path.join(os.getcwd(), "birdmaps"))
from birdmaps import bird_runs


## Bird Run Definition

In [28]:
birds_for_paper = []

bird_run = bird_runs.BirdRun(DATA_PATH)

birds = []

NUM_SIMULATIONS = 400
RUN_NAME = "Paper"

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    do_validation=False, run_name=RUN_NAME,
    hop_distance=2, num_spreads=20,
    num_simulations=NUM_SIMULATIONS))

# Steller's Jay, hop distance = 2, num spreads = 4
birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    do_validation=False, run_name=RUN_NAME,
    hop_distance=2, num_spreads=3,
    num_simulations=NUM_SIMULATIONS))

In [29]:

california = GeoTiff.from_file(os.path.join(DATA_PATH, "Terrain", "california_land.tif"))
cal_tile = california.get_all_as_tile()

In [30]:
def compute_fca(bird):
    hab = GeoTiff.from_file(bird.habitat_fn)
    rep = GeoTiff.from_file(bird.repopulation_fn)
    hab_tile = hab.get_all_as_tile()
    rep_tile = rep.get_all_as_tile()
    habitat = np.sum(hab_tile.m * cal_tile.m) * 0.3 * 0.3 
    fca = np.sum(rep_tile.m * cal_tile.m / 255.) * 0.3 * 0.3
    connected_fraction = np.sum(rep_tile.m * cal_tile.m / 255.) / np.sum(hab_tile.m * cal_tile.m)
    print("{} habitat: {} FCA: {} connected fraction: {}".format(bird.name, habitat, fca, connected_fraction))
    return fca

In [31]:
for bird in birds:
    compute_fca(bird)

Acorn Woodpecker habitat: 66988.43999999999 FCA: 50684.38305882348 connected fraction: 0.7566138733611871
Steller's Jay habitat: 77716.43999999999 FCA: 69713.48964705862 connected fraction: 0.8970237139922858


Let us estimate the percentage difference from using 400 vs. 10000 simulations for Acorn Woodpecker (for Steller's Jay it's much lower).

In [32]:
fca10000 = compute_fca(birds[0])

acowoo400 = bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    do_validation=False, run_name="Paper10000",
    hop_distance=2, num_spreads=20,
    num_simulations=10000)
fca400 = compute_fca(acowoo400)

100 * abs(fca10000 - fca400) / min(fca10000, fca400)

Acorn Woodpecker habitat: 66988.43999999999 FCA: 50684.38305882348 connected fraction: 0.7566138733611871
Acorn Woodpecker habitat: 66988.43999999999 FCA: 50798.99541176463 connected fraction: 0.7583248006934424


0.2261295216085231